In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
from PIL import Image
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import image as mp_image
import seaborn as sns
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Transforms (adds variation in images to help generalise the CNN model)**

In [3]:
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

**Load data set then split images into training and validation sets**

In [4]:
dataset = torchvision.datasets.ImageFolder(r'../input/mayo-compressed/MAYO', transform=transformer)
print(len(dataset))

753


In [5]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [6]:
datasets = train_val_dataset(dataset)
train_count = len(datasets['train'])
val_count = len(datasets['val'])
# summary = datasets['train'].dataset

print(train_count)
print(val_count)
# print(summary)                 

564
189


In [7]:
train_loader = DataLoader(datasets['train'], batch_size=32, shuffle=True)
val_loader = DataLoader(datasets['val'], batch_size=32, shuffle=True)
print(train_loader)

**CNN Architecture and feed forward function**

In [8]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
    
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
    def forward(self,input):
        
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
                
        output = output.view(-1,32*75*75)
               
        output = self.fc(output)
            
        return output
            
        

**Instantiate CNN model**

In [9]:
model = ConvNet(num_classes=2).to(device)

**Instantiate optmizer and loss function**

In [10]:
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

**Hyperparameter: number of epochs**

In [11]:
num_epochs = 10

**Training the CNN model and saving the parameters that give the highest validation set accuracy**

In [12]:
best_accuracy = 0.0

for epoch in range(num_epochs):
    
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss += loss.cpu().data * images.size(0)
        _,prediction = torch.max(outputs.data,1)
        
        train_accuracy += int(torch.sum(prediction==labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    
    model.eval()
    
    val_accuracy = 0.0
    for i, (images,labels) in enumerate(val_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _,prediction = torch.max(outputs.data,1)
        val_accuracy += int(torch.sum(prediction==labels.data))
    
    val_accuracy = val_accuracy/val_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Val Accuracy: '+str(val_accuracy))
    
    
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(),'checkpoint.pth')
        best_accuracy = val_accuracy

Epoch: 0 Train Loss: tensor(18.1959) Train Accuracy: 0.5886524822695035 Val Accuracy: 0.6931216931216931
Epoch: 1 Train Loss: tensor(4.0556) Train Accuracy: 0.5886524822695035 Val Accuracy: 0.4444444444444444
Epoch: 2 Train Loss: tensor(2.4163) Train Accuracy: 0.6453900709219859 Val Accuracy: 0.5079365079365079
Epoch: 3 Train Loss: tensor(3.0604) Train Accuracy: 0.601063829787234 Val Accuracy: 0.7354497354497355
Epoch: 4 Train Loss: tensor(4.8770) Train Accuracy: 0.5939716312056738 Val Accuracy: 0.30158730158730157
Epoch: 5 Train Loss: tensor(3.1904) Train Accuracy: 0.6436170212765957 Val Accuracy: 0.6825396825396826
Epoch: 6 Train Loss: tensor(3.2809) Train Accuracy: 0.6134751773049646 Val Accuracy: 0.7354497354497355
Epoch: 7 Train Loss: tensor(3.1565) Train Accuracy: 0.6578014184397163 Val Accuracy: 0.5714285714285714
Epoch: 8 Train Loss: tensor(1.3374) Train Accuracy: 0.6790780141843972 Val Accuracy: 0.5608465608465608
Epoch: 9 Train Loss: tensor(1.6515) Train Accuracy: 0.645390070

In [ ]:
'''
Epoch: 0 Train Loss: tensor(18.1959) Train Accuracy: 0.5886524822695035 Val Accuracy: 0.6931216931216931
Epoch: 1 Train Loss: tensor(4.0556) Train Accuracy: 0.5886524822695035 Val Accuracy: 0.4444444444444444
Epoch: 2 Train Loss: tensor(2.4163) Train Accuracy: 0.6453900709219859 Val Accuracy: 0.5079365079365079
Epoch: 3 Train Loss: tensor(3.0604) Train Accuracy: 0.601063829787234 Val Accuracy: 0.7354497354497355
Epoch: 4 Train Loss: tensor(4.8770) Train Accuracy: 0.5939716312056738 Val Accuracy: 0.30158730158730157
Epoch: 5 Train Loss: tensor(3.1904) Train Accuracy: 0.6436170212765957 Val Accuracy: 0.6825396825396826
Epoch: 6 Train Loss: tensor(3.2809) Train Accuracy: 0.6134751773049646 Val Accuracy: 0.7354497354497355
Epoch: 7 Train Loss: tensor(3.1565) Train Accuracy: 0.6578014184397163 Val Accuracy: 0.5714285714285714
Epoch: 8 Train Loss: tensor(1.3374) Train Accuracy: 0.6790780141843972 Val Accuracy: 0.5608465608465608
Epoch: 9 Train Loss: tensor(1.6515) Train Accuracy: 0.6453900709219859 Val Accuracy: 0.708994708994709

'''